# 컴피티션 링크
- https://www.kaggle.com/t/2e45abe9f1434b59a3358365432a48bb

# 고객별 백화점 구매기록 데이터셋
- 미혼(1) or 기혼(0) 고객을 예측
- 학습용 구매기록 데이터
    - https://drive.google.com/file/d/1tg41qXu02FK55bFa8P1Gx2URhwPUCoQQ/view?usp=sharing
- 학습용 정답 데이터
    - https://drive.google.com/file/d/1-9AcU9nAoO4SzSmqdCOYTvWNpA5Pdt3q/view?usp=sharing
- 테스트용 구매기록 데이터
    - https://drive.google.com/file/d/1-AwDfGlHm9rNtpnHIWOK96jBJYy3f2SZ/view?usp=sharing
- 제출 양식 데이터
    - https://drive.google.com/file/d/1-Qv7SlsY5Eu3bRR7Z0IUJyLkOO1Fl3y6/view?usp=sharing

In [377]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 데이터 경로 변수

In [378]:
DATA_PATH = "/content/drive/MyDrive/data/"
DATA_PATH

'/content/drive/MyDrive/data/'

- 데이터 불러오기

In [379]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [380]:
# from IPython.display import display, HTML
# br_html = HTML("<br><br>")
# display(HTML("<h1>학습용 구매기록 데이터</h1>"), train_tr.head(), br_html)
# display(HTML("<h1>학습용 정답 데이터</h1>"), train_target.head(), br_html)
# display(HTML("<h1>테스트용 구매기록 데이터</h1>"), test_tr.head(), br_html)
# display(HTML("<h1>제출 양식 데이터</h1>"), submit.head(), br_html)

In [381]:
# display(HTML("<h1>학습용 구매기록 데이터</h1>"))
# display(train_tr.info(), br_html)

# display(HTML("<h1>학습용 정답 데이터</h1>"))
# display(train_target.info(), br_html)

# display(HTML("<h1>테스트용 구매기록 데이터</h1>"))
# display(test_tr.info(), br_html)

# display(HTML("<h1>제출 양식 데이터</h1>"))
# display(submit.info(), br_html)

In [382]:
pd.concat([train_tr, train_target], axis = 1)

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격,ID,target
0,train_13219,2004-05-01 09:40:00,A144000,공산품파트,차류,5100,59700,train_0,1.0
1,train_5590,2004-05-01 09:40:00,A144000,잡화파트,화장잡화,5101,17000,train_1,1.0
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937,train_2,0.0
3,train_3010,2004-05-01 10:30:00,A373000,아동_스포츠,아동복,5105,19000,train_3,0.0
4,train_10851,2004-05-01 10:30:00,A112000,가정용품,전화기_카세트,5110,215000,train_4,0.0
...,...,...,...,...,...,...,...,...,...
523100,train_330,2005-04-29 19:53:00,A112000,영캐릭터,트렌드캐주얼,5882,484000,NaN,NaN
523101,train_9738,2005-04-29 19:54:00,A373000,아동_스포츠,상품군미지정,5191,144000,NaN,NaN
523102,train_10787,2005-04-29 20:00:00,A112000,공산품,음료,5100,8340,NaN,NaN
523103,train_13129,2005-04-29 20:00:00,A112000,생식품,용기보증,5100,23170,NaN,NaN


# 특성 공학(Feature Engineering)

## 날짜 형식으로 변환

In [383]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])
train_tr["구매일시"].max()

Timestamp('2005-04-29 20:03:00')

## 새로 만든 feature와 병합할 고객ID로만 이루어진 데이터프레임 생성

In [384]:
train_ft = train_target[["ID"]]
test_ft = submit[["ID"]]

train_ft.shape, test_ft.shape

((14940, 1), (12225, 1))

In [385]:
train_tr['중분류'].values[:]

array(['차류', '화장잡화', '용기보증', ..., '음료', '용기보증', '베이직캐주얼'], dtype=object)

## 구매일시를 이용한 특성생성

In [386]:
# 저녁 시간18~21, 5월4일~5월5일, 어버이날

In [387]:
agg_list = [
        # 컬럼명, 집계 방식
        ('내점일수','nunique'),
        ('구매주기', lambda x: int( (x.max() - x.min()).days / x.dt.date.nunique()) ),
        ('주말방문비율', lambda x: np.mean(x.dt.weekday>4)),
        ('봄_구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
        ('여름_구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
        ('가을_구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
        ('겨울_구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12]))),
        ('주구매요일', lambda x: x.dt.weekday.mode()[0]),
        ('일별평균구매건수', lambda x:  x.count() / x.dt.date.nunique() ),
        ('거래개월수', lambda x: x.dt.date.astype(str).str[:-3].nunique() ),
    ]

tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,7
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,11
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,12
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,12
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,10


In [388]:
tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,4
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,3
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,2
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,7
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,4


## 지점을 이용한 특성생성

In [389]:
agg_list = [
          ("방문지점수","nunique"),
          ('주구매지점', lambda x: x.mode()[0]),
    ]

tmp = train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,방문지점수,주구매지점
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,7,2,A144000
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,11,3,A112000
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,12,2,A373000
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,12,3,A144000
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,10,2,A144000


In [390]:
tmp = test_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,방문지점수,주구매지점
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,4,1,A202000
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,3,2,A373000
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,2,1,A144000
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,7,1,A112000
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,4,3,A112000


## 브랜드코드를 이용한 특성생성

In [391]:
agg_list = [
             ('브랜드코드_nunique', 'nunique'),
             ]

tmp = train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,방문지점수,주구매지점,브랜드코드_nunique
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,7,2,A144000,13
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,11,3,A112000,26
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,12,2,A373000,58
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,12,3,A144000,99
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,10,2,A144000,23


In [392]:
tmp = test_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,방문지점수,주구매지점,브랜드코드_nunique
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,4,1,A202000,5
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,3,2,A373000,4
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,2,1,A144000,5
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,7,1,A112000,19
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,4,3,A112000,13


## 중분류를 이용한 특성생성

In [393]:
agg_list = [
            ('중분류_nunique', 'nunique'),
            ('주구매_중분류', lambda x: x.mode()[0]),
            ]
tmp = train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,방문지점수,주구매지점,브랜드코드_nunique,중분류_nunique,주구매_중분류
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,7,2,A144000,13,11,골프웨어
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,11,3,A112000,26,24,영캐주얼
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,12,2,A373000,58,35,아웃도어
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,12,3,A144000,99,60,수입종합화장품
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,10,2,A144000,23,24,로얄부틱2F


In [394]:
tmp = test_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,방문지점수,주구매지점,브랜드코드_nunique,중분류_nunique,주구매_중분류
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,4,1,A202000,5,4,용기보증
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,3,2,A373000,4,4,남성구두
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,2,1,A144000,5,4,수입종합화장품
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,7,1,A112000,19,12,수입종합화장품
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,4,3,A112000,13,12,칼라드래디셔널


## 대분류를 이용한 특성생성

In [395]:
agg_list = [
            ('대분류_nunique', 'nunique'),
            ('주구매_대분류', lambda x: x.mode()[0]),
            ]

tmp = train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,방문지점수,주구매지점,브랜드코드_nunique,중분류_nunique,주구매_중분류,대분류_nunique,주구매_대분류
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,7,2,A144000,13,11,골프웨어,9,남성정장스포츠
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,11,3,A112000,26,24,영캐주얼,12,영플라자
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,12,2,A373000,58,35,아웃도어,14,명품잡화
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,12,3,A144000,99,60,수입종합화장품,16,케주얼_구두_아동
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,10,2,A144000,23,24,로얄부틱2F,8,명품잡화


In [396]:
tmp = test_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,방문지점수,주구매지점,브랜드코드_nunique,중분류_nunique,주구매_중분류,대분류_nunique,주구매_대분류
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,4,1,A202000,5,4,용기보증,5,공산품
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,3,2,A373000,4,4,남성구두,3,명품잡화
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,2,1,A144000,5,4,수입종합화장품,3,잡화파트
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,7,1,A112000,19,12,수입종합화장품,5,영플라자
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,4,3,A112000,13,12,칼라드래디셔널,6,여성캐주얼


## 구매가격을 이용한 특성생성

In [397]:
agg_list = [
        ('총구매액','sum'),
        ('구매건수', 'count'),
        ('평균구매액', "mean"),
        ('최대구매액', 'max'),
        ('총구매액_왜도', 'skew'),
        ('최소구매액',lambda x: x[x > 0].min() ) ,
        ('환불금액',lambda x: x[x < 0].sum() ) ,
        ('환불건수', lambda x: ( x < 0 ).sum() ),
        ('구매금액표준편차',lambda x: x[x>0].std() )
    ]

tmp = train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,주구매_대분류,총구매액,구매건수,평균구매액,최대구매액,총구매액_왜도,최소구매액,환불금액,환불건수,구매금액표준편차
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,남성정장스포츠,4201200,20,210060.000000,1236000,-0.656466,20000,-2517000,3,403660.245259
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,영플라자,5043025,42,120072.023810,698000,2.651821,5600,0,0,126592.626144
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,명품잡화,22135386,114,194170.052632,2770000,4.375304,4400,-1072500,7,406609.265932
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,케주얼_구두_아동,35594762,211,168695.554502,1492000,1.794671,10000,-2867800,14,236976.860907
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,명품잡화,3988866,62,64336.548387,1204000,-0.260189,7640,-6954400,13,308503.944915


In [398]:
tmp = test_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,주구매_대분류,총구매액,구매건수,평균구매액,최대구매액,총구매액_왜도,최소구매액,환불금액,환불건수,구매금액표준편차
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,공산품,429579,7,61368.428571,110000,0.702551,26643,0,0,32293.148313
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,명품잡화,290600,4,72650.000000,118000,0.856705,37600,0,0,33658.431336
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,잡화파트,607000,5,121400.000000,326000,1.969409,38000,0,0,117233.954126
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,영플라자,2305740,32,72054.375000,403000,-0.614857,7040,-1092000,4,122556.774972
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,여성캐주얼,1568690,21,74699.523810,376000,0.182857,16200,-372200,3,94997.232393


In [399]:
train_ft.shape, test_ft.shape

((14940, 27), (12225, 27))

## pivot_table을 이용한 특성 생성

In [400]:
# train_tmp = pd.pivot_table(train_tr,index="ID",columns="중분류",values="구매가격",aggfunc="count",fill_value=0).add_prefix("pivot_cnt_").reset_index()
# train_ft = train_ft.merge(train_tmp,how="left",on="ID")
# train_ft.head()

In [401]:
# test_tmp = pd.pivot_table(test_tr,index="ID",columns="중분류",values="구매가격",aggfunc="count",fill_value=0).add_prefix("pivot_cnt_").reset_index()
# test_tmp.head()

In [402]:
# for col in train_tmp.columns:
#     if col not in test_tmp.columns:
#         test_tmp[col] = 0

# test_tmp = test_tmp[train_tmp.columns]

In [403]:
# test_ft = test_ft.merge(test_tmp,how="left",on="ID")
# test_ft.head()

In [404]:
# cols = [ col for col in train_ft.columns if col.startswith("pivot_cnt_") ]

In [405]:
# train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
# train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
# train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

# test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
# test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
# test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

# train_ft.shape, test_ft.shape

## 구매일시 활용

In [406]:
# 저녁 시간18~21, 5월4일~5월5일, 어버이날

In [407]:
((train_tr["구매일시"].dt.hour > 18) & (train_tr["구매일시"].dt.hour < 21)).mean()

0.0899953164278682

In [408]:
display(train_tr["구매일시"].max())
display(train_tr["구매일시"].min())
display(train_tr["구매일시"].min() + pd.Timedelta(days=3))
display(train_tr["구매일시"].min() + pd.Timedelta(days=7))

Timestamp('2005-04-29 20:03:00')

Timestamp('2004-05-01 09:40:00')

Timestamp('2004-05-04 09:40:00')

Timestamp('2004-05-08 09:40:00')

In [409]:
train_tr["구매일시"].dt.month == 5

,구매일시
0,True
1,True
2,True
3,True
4,True
...,...
523100,False
523101,False
523102,False
523103,False


In [410]:
(train_tr["구매일시"].dt.day == 4).value_counts()

,count
구매일시,
False,504528
True,18577


In [411]:
np.mean((train_tr["구매일시"].dt.month == 5) & (train_tr["구매일시"].dt.day == 4))

0.0030357194062377536

In [412]:
# 날짜 기준 ID별 5/5 전날 > groupby 구매비용 + 아동 포함된 구매비용

In [413]:
# # code_test
# agg_list = [
#         # 컬럼명, 집계 방식
#         ('퇴근후_구매비율', lambda x: np.mean((x.dt.hour >= 18) & (x.dt.hour <= 21))),
#         ('2004_어린이날_전날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 4))), # 아동관련 구매비율
#         ('2004_어린이날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 5))),
#         ('2004_어버이날_전날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 7))),
#         ('2004_어버이날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 8))),
#     ]
# train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()

In [414]:
#code_acting
agg_list = [
        # 컬럼명, 집계 방식
        ('퇴근후_구매비율', lambda x: np.mean((x.dt.hour >= 18) & (x.dt.hour <= 21))),
        ('2004_어린이날_전날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 4))), # 아동관련 구매비율
        ('2004_어린이날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 5))),
        ('2004_어버이날_전날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 7))),
        ('2004_어버이날_구매비율', lambda x : np.mean((x.dt.month == 5) & (x.dt.day == 8))),
    ]

tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,총구매액_왜도,최소구매액,환불금액,환불건수,구매금액표준편차,퇴근후_구매비율,2004_어린이날_전날_구매비율,2004_어린이날_구매비율,2004_어버이날_전날_구매비율,2004_어버이날_구매비율
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,-0.656466,20000,-2517000,3,403660.245259,0.250000,0.0,0.0,0.05,0.0
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,2.651821,5600,0,0,126592.626144,0.309524,0.0,0.0,0.00,0.0
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,4.375304,4400,-1072500,7,406609.265932,0.149123,0.0,0.0,0.00,0.0
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,1.794671,10000,-2867800,14,236976.860907,0.222749,0.0,0.0,0.00,0.0
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,-0.260189,7640,-6954400,13,308503.944915,0.161290,0.0,0.0,0.00,0.0


In [415]:
tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,총구매액_왜도,최소구매액,환불금액,환불건수,구매금액표준편차,퇴근후_구매비율,2004_어린이날_전날_구매비율,2004_어린이날_구매비율,2004_어버이날_전날_구매비율,2004_어버이날_구매비율
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,0.702551,26643,0,0,32293.148313,0.000000,0.0,0.0,0.0,0.0
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,0.856705,37600,0,0,33658.431336,0.000000,0.0,0.0,0.0,0.0
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,1.969409,38000,0,0,117233.954126,0.000000,0.0,0.0,0.0,0.0
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,-0.614857,7040,-1092000,4,122556.774972,0.531250,0.0,0.0,0.0,0.0
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,0.182857,16200,-372200,3,94997.232393,0.571429,0.0,0.0,0.0,0.0


In [416]:
train_ft.shape, test_ft.shape

((14940, 32), (12225, 32))

## 지점 활용

In [417]:
# 전부 기혼 우세

In [418]:
# train_tr['지점코드'].value_counts()

In [419]:
# train_tr['지점코드'].unique()

In [420]:
# #code test
# agg_list = [
#           ('', ),
#     ]
# train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()

In [421]:
# # code_acting
# agg_list = [
#           ('', ),
#     ]

# tmp = train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [422]:
# tmp = test_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

## 브랜드코드 활용

In [423]:
# 5100, 5110, 5111, 5112

In [424]:
# train_tr['브랜드코드'].value_counts()

In [425]:
# train_tr.groupby('ID')["브랜드코드"].count()

In [426]:
# # code test
# agg_list = [
#              ('', ),
#              ]
# train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()

In [427]:
# # code acting
# agg_list = [
#              ('', ),
#              ]

# tmp = train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
# train_ft = train_ft.merge(tmp, how='left',on="ID")
# train_ft.head()

In [428]:
# tmp = test_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
# test_ft = test_ft.merge(tmp, how='left',on="ID")
# test_ft.head()

## 중분류 활용

In [429]:
# 중분류별 구매횟수 평균mean 표준편차std 왜도skew 첨도kurt

In [430]:
# 가전제품 : 가스기기, 가스렌지, 가전특정, 냉장고_세탁기, 냉장고.세탁기, TV_VTR, 냉난방 > ID별 모든중분류 구매액에서 해당 상품(전체) 구매한 비율 + 모든중분류 금액 비율
# 공산_식품 : 건강식품 건식품
# 생식품 : 곡물 생선 야채 정육 청과
# 여성_패션 : 가방 란제리 국내부띠끄 로얄부틱2F 명품 모피 모피_피혁 모피.피혁 모피_니트 미시케쥬얼 보석 부띠끄 수입부띠끄 수입종합화장품 엘레강스 엘레강스부틱 엘레강스캐주얼 타운웨어 크리스탈
# 남성_패션 : 트래디셔널 내셔날 남성라이센스
# 스포츠 : 골프(국내), 골프(수입), 골프(LC), 골프(NB), 골프 단품, 골프웨어

In [431]:
train_tr['중분류'].nunique()

303

In [432]:
all_middle_list = train_tr['중분류'].unique().tolist()
len(all_middle_list)

303

In [433]:
# price = []
# for all_middle in all_middle_list:
#   price.append(train_tr[train_tr['중분류'] == all_middle]['구매가격'].sum())

# sum(price)

In [434]:
train_tr['구매가격'].sum()

51648607528

In [435]:
# # ID별 모든중분류 구매가격
# train_tr.groupby(['ID', '중분류'])['구매가격'].sum().reset_index().head(30)

In [436]:
# # ID별 모든중분류 구매액 > ID별 중분류에 대한 총 구매가격
# (train_tr.groupby(['ID', '중분류'])['구매가격'].sum().reset_index()).groupby('ID')['구매가격'].sum().reset_index()

In [437]:
# 중분류_기혼우세
elecs = ['가스기기', '가스렌지', '가전특정', '냉장고_세탁기', '냉장고.세탁기', 'TV_VTR', '냉난방']
fac_foods = ['건강식품', '건식품']
uncooked_foods = ['곡물', '생선', '야채', '정육', '청과']
fe_fashions = ['가방', '란제리', '국내부띠끄', '로얄부틱2F', '명품', '모피', '모피_피혁', '모피.피혁,' '모피_니트', '미시케쥬얼', '보석', '부띠끄', '수입부띠끄', '수입종합화장품', '엘레강스', '엘레강스부틱', '엘레강스캐주얼', '타운웨어', '크리스탈']
ma_fashions = ['트래디셔널', '내셔날', '남성라이센스']
sports = ['골프(국내)', '골프(수입)', '골프(LC)', '골프(NB)', '골프단품', '골프웨어']
homewares = ['브랜드침구', '주방용품', '침구']

In [438]:
# 질문 > ID별 중분류 구매가격 합에서 가전제품, 공산_식품 구매 비율 구하기
# # ID별 중분류별 구매가격 합계
# grouped = df.groupby(['ID', '중분류'])['구매가격'].sum().reset_index()

# # 가전제품, 공산_식품 필터링
# target_categories = ['가전제품', '공산_식품']
# filtered_df = grouped[grouped['중분류'].isin(target_categories)]

# # ID별 총 구매가격
# total_by_id = grouped.groupby('ID')['구매가격'].sum()

# # ID별 목표 카테고리 구매 비율 계산
# result_df = filtered_df.groupby('ID')['구매가격'].sum().div(total_by_id, level='ID').reset_index()

In [439]:
# train_tr.groupby(['ID', '중분류'])['구매가격'].sum().reset_index()[train_tr.groupby(['ID', '중분류'])['구매가격'].sum().reset_index()['중분류'].isin(fe_fashions)].groupby('ID')['구매가격'].sum().div(train_tr.groupby(['ID', '중분류'])['구매가격'].sum(), level='ID').reset_index().head(50)

In [440]:
# # ID별 모든중분류에서 가전 제품에 해당되는 value가 있는지 확인
# train_tr.groupby(['ID', '중분류'])['구매가격'].sum().reset_index().groupby('ID')['중분류'].count()

In [441]:
# # 가전 제품 해당 value 총 금액
# elec_price = []
# for elec in elecs:
#   price.append(train_tr[train_tr['중분류'] == elec ]['구매가격'].sum())

# sum(elec_price)

In [442]:
# # 공산_식품 해당 value 총 금액
# fac_price = []
# for fac_food in fac_foods:
#   price.append(train_tr[train_tr['중분류'] == fac_food ]['구매가격'].sum())

# sum(fac_price)

In [443]:
# # 생식품 해당 value 총 금액
# uncooked_price = []
# for uncooked_food in uncooked_foods:
#   price.append(train_tr[train_tr['중분류'] == uncooked_food ]['구매가격'].sum())

# sum(uncooked_price)

In [444]:
# # 여성패션 해당 value 총 금액
# fe_price = []
# for fe_fashion in fe_fashions:
#   price.append(train_tr[train_tr['중분류'] == fe_fashion ]['구매가격'].sum())

# sum(uncooked_price)

In [445]:
# # 남성패션 해당 value 총 금액
# ma_price = []
# for ma_fashion in ma_fashions:
#   price.append(train_tr[train_tr['중분류'] == ma_fashion ]['구매가격'].sum())

# sum(ma_price)

In [446]:
# # 스포츠 해당 value 총 금액
# sport_price = []
# for sport in sports:
#   price.append(train_tr[train_tr['중분류'] == sport ]['구매가격'].sum())

# sum(sport_price)

In [447]:
# # 가정용품 해당 value 총 금액
# homeware_price = []
# for homeware in homewares:
#   price.append(train_tr[train_tr['중분류'] == homeware ]['구매가격'].sum())

# sum(homeware_price)

In [448]:
train_tr['중분류'].str.startswith('가').astype(int)

,중분류
0,0
1,0
2,0
3,0
4,0
...,...
523100,0
523101,0
523102,0
523103,0


In [449]:
train_tr['중분류'].str.endswith('식품').astype(int)

,중분류
0,0
1,0
2,0
3,0
4,0
...,...
523100,0
523101,0
523102,0
523103,0


In [450]:
train_tr['중분류'].isin(['곡물', '생선', '야채', '정육', '청과']).astype(int)

,중분류
0,0
1,0
2,0
3,0
4,0
...,...
523100,0
523101,0
523102,0
523103,0


In [451]:
fe_cols = '가방', '란제리', '국내부띠끄', '로얄부틱2F', '명품 모피', '모피_피혁', '모피.피혁', '모피_니트', '미시케쥬얼', '보석', '부띠끄', '수입부띠끄', '수입종합화장품', '엘레강스', '엘레강스부틱', '엘레강스캐주얼', '타운웨어'

In [452]:
train_tr['중분류'].isin(fe_cols).astype(int)

,중분류
0,0
1,0
2,0
3,0
4,0
...,...
523100,0
523101,0
523102,0
523103,0


In [453]:
ma_cols = '트래디셔널', '내셔날', '남성라이센스'

In [454]:
train_tr['중분류'].isin(ma_cols).astype(int)

,중분류
0,0
1,0
2,0
3,0
4,0
...,...
523100,0
523101,0
523102,0
523103,0


In [455]:
train_tr.groupby('ID')['중분류'].nunique()

,중분류
ID,
train_0,11
train_1,24
train_10,29
train_100,14
train_1000,12
...,...
train_9995,1
train_9996,15
train_9997,9


In [456]:
train_tr.groupby('ID')['중분류'].count()

,중분류
ID,
train_0,20
train_1,42
train_10,134
train_100,26
train_1000,45
...,...
train_9995,1
train_9996,25
train_9997,27


In [457]:
# # code_test
# fe_cols = '가방', '란제리', '국내부띠끄', '로얄부틱2F', '명품 모피', '모피_피혁', '모피.피혁', '모피_니트', '미시케쥬얼', '보석', '부띠끄', '수입부띠끄', '수입종합화장품', '엘레강스', '엘레강스부틱', '엘레강스캐주얼', '타운웨어'
# ma_cols = '트래디셔널', '내셔날', '남성라이센스'
# agg_list = [
#             ('중분류_방문횟수_평균', lambda x: x.count() / x.nunique()),
#             ('가전제품', lambda x: np.mean(x.str.startswith('가').astype(int))),
#             ('공산_식품', lambda x: np.mean(x.str.endswith('식품').astype(int))),
#             ('생식품', lambda x: np.mean(x.isin(['곡물', '생선', '야채', '정육', '청과']).astype(int))),
#             ('여성_기혼_패션', lambda x: np.mean(x.isin(fe_cols).astype(int))),
#             ('남성_기혼_패션', lambda x: np.mean(x.isin(ma_cols).astype(int)))

#             ]
# train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index().head(60)

In [458]:
# code_acting
agg_list = [
            ('중분류_방문횟수_평균', lambda x: x.count() / x.nunique()),
            ('가전제품', lambda x: np.mean(x.str.startswith('가').astype(int))),
            ('공산_식품', lambda x: np.mean(x.str.endswith('식품').astype(int))),
            ('생식품', lambda x: np.mean(x.isin(['곡물', '생선', '야채', '정육', '청과']).astype(int))),
            ('여성_기혼_패션', lambda x: np.mean(x.isin(fe_cols).astype(int))),
            ('남성_기혼_패션', lambda x: np.mean(x.isin(ma_cols).astype(int)))
            ]
tmp = train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,2004_어린이날_전날_구매비율,2004_어린이날_구매비율,2004_어버이날_전날_구매비율,2004_어버이날_구매비율,중분류_방문횟수_평균,가전제품,공산_식품,생식품,여성_기혼_패션,남성_기혼_패션
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,0.0,0.0,0.05,0.0,1.818182,0.000000,0.000000,0.050000,0.050000,0.000000
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,0.0,0.0,0.00,0.0,1.750000,0.000000,0.023810,0.023810,0.214286,0.023810
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,0.0,0.0,0.00,0.0,3.257143,0.008772,0.000000,0.000000,0.263158,0.043860
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,0.0,0.0,0.00,0.0,3.516667,0.004739,0.009479,0.004739,0.270142,0.033175
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,0.0,0.0,0.00,0.0,2.583333,0.000000,0.016129,0.048387,0.370968,0.000000


In [459]:
tmp = test_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,2004_어린이날_전날_구매비율,2004_어린이날_구매비율,2004_어버이날_전날_구매비율,2004_어버이날_구매비율,중분류_방문횟수_평균,가전제품,공산_식품,생식품,여성_기혼_패션,남성_기혼_패션
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,0.0,0.0,0.0,0.0,1.750000,0.0,0.000000,0.0,0.285714,0.000000
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.250000,0.000000
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,0.0,0.0,0.0,0.0,1.250000,0.0,0.000000,0.0,0.600000,0.200000
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,0.0,0.0,0.0,0.0,2.666667,0.0,0.000000,0.0,0.250000,0.000000
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,0.0,0.0,0.0,0.0,1.750000,0.0,0.047619,0.0,0.095238,0.047619


In [460]:
train_ft.shape, test_ft.shape

((14940, 38), (12225, 38))

##대분류 활용

In [461]:
# 대분류 : 아동+청소년+가정용품, 생식품+아동 4분위
# 대분류
#   가정용품, 골프_유니캐주얼, 공산품, 남성의류, 남성정장스포츠, 로얄부띠끄, 로얄부틱
#   명품잡화, 생식품, 스포츠캐쥬얼, 여성의류파트, 여성정장, 여성캐주얼, 여성캐쥬얼
#   잡화, 잡화파트, 패션잡화

In [462]:
train_tr['대분류'].unique()

array(['공산품파트', '잡화파트', '공산품', '아동_스포츠', '가정용품', '아동문화', '케주얼_구두_아동',
       '스포츠캐쥬얼', '아동', '패션잡화', '여성캐쥬얼', '생식품파트', '영어덜트캐쥬얼', '로얄부띠끄',
       '생식품', '남성정장스포츠', '명품잡화', '골프_유니캐쥬얼', '남성의류', '여성정장', '잡화',
       '여성의류파트', '영라이브', '여성캐주얼', '영플라자', '가정용품파트', '로얄부틱', '영캐릭터'],
      dtype=object)

In [463]:
train_tr['대분류'].isin(['가정용품','골프_유니캐주얼','생식품'])

,대분류
0,False
1,False
2,False
3,False
4,True
...,...
523100,False
523101,False
523102,False
523103,True


In [464]:
cols = ['가정용품', '골프_유니캐주얼', '로얄부띠끄', '로얄부틱', '명품잡화', '생식품']

In [465]:
train_tr['대분류'].isin(cols).astype(int)

,대분류
0,0
1,0
2,0
3,0
4,1
...,...
523100,0
523101,0
523102,0
523103,1


In [466]:
(train_tr['대분류'].str.startswith('아','영'))

,대분류
0,False
1,False
2,False
3,True
4,False
...,...
523100,False
523101,True
523102,False
523103,False


In [467]:
train_tr['대분류'].str.endswith('아동').value_counts()

,count
대분류,
False,475118
True,47987


In [468]:
((train_tr['대분류'].str.startswith('아','영')) & (train_tr['대분류'].str.endswith('아동'))).astype(int)

,대분류
0,0
1,0
2,0
3,0
4,0
...,...
523100,0
523101,0
523102,0
523103,0


In [469]:
# # code_test
# agg_list = [
#             ('아동_대분류', lambda x : np.mean(((x.str.startswith('아')) & (x.str.endswith('아동'))).astype(int))),
#             ('여성_기혼', lambda x: np.mean((x.str.startswith('명품','로얄')).astype(int))),
#             ('남녀_기혼', lambda x: np.mean(x.isin(['가정용품','골프_유니캐주얼','생식품']))),


#             ]
# train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()

In [470]:
# code acting
agg_list = [
            ('아동_대분류', lambda x : np.mean(((x.str.startswith('아')) & (x.str.endswith('아동'))).astype(int))),
            ('여성_기혼', lambda x: np.mean((x.str.startswith('명품','로얄')).astype(int))),
            ('남녀_기혼', lambda x: np.mean(x.isin(['가정용품','골프_유니캐주얼','생식품']))),
            ('영_아동_대분류', lambda x : np.mean(((x.str.startswith('아')) & (x.str.endswith('아동'))).astype(int)))
            ]

tmp = train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,중분류_방문횟수_평균,가전제품,공산_식품,생식품,여성_기혼_패션,남성_기혼_패션,아동_대분류,여성_기혼,남녀_기혼,영_아동_대분류
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,1.818182,0.000000,0.000000,0.050000,0.050000,0.000000,0.0,0.000000,0.150000,0.0
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,1.750000,0.000000,0.023810,0.023810,0.214286,0.023810,0.0,0.047619,0.023810,0.0
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,3.257143,0.008772,0.000000,0.000000,0.263158,0.043860,0.0,0.289474,0.052632,0.0
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,3.516667,0.004739,0.009479,0.004739,0.270142,0.033175,0.0,0.018957,0.000000,0.0
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,2.583333,0.000000,0.016129,0.048387,0.370968,0.000000,0.0,0.241935,0.000000,0.0


In [471]:
tmp = test_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,중분류_방문횟수_평균,가전제품,공산_식품,생식품,여성_기혼_패션,남성_기혼_패션,아동_대분류,여성_기혼,남녀_기혼,영_아동_대분류
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,1.750000,0.0,0.000000,0.0,0.285714,0.000000,0.285714,0.0,0.142857,0.285714
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,1.000000,0.0,0.000000,0.0,0.250000,0.000000,0.000000,0.5,0.000000,0.000000
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,1.250000,0.0,0.000000,0.0,0.600000,0.200000,0.000000,0.0,0.000000,0.000000
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,2.666667,0.0,0.000000,0.0,0.250000,0.000000,0.000000,0.0,0.031250,0.000000
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,1.750000,0.0,0.047619,0.0,0.095238,0.047619,0.000000,0.0,0.000000,0.000000


In [472]:
train_ft.shape, test_ft.shape

((14940, 42), (12225, 42))

## 구매가격 활용

In [473]:
# 상위 10% > vip,
# 왜도+첨도
vip_num = int(train_tr.groupby('ID')['구매가격'].mean().count() / 10)

In [474]:
train_tr.groupby('ID')["구매가격"].sum().sort_values(ascending = False)

,구매가격
ID,
train_11204,99190685
train_12338,85098557
train_3924,82957000
train_11804,82094852
train_12262,72067270
...,...
train_9347,-131000
train_9102,-404945
train_13327,-727800


In [475]:
vip_num = int(train_tr.groupby('ID')['구매가격'].mean().count() * 0.1)
vip_num
pd.Series(train_tr.groupby('ID')["구매가격"].sum()).sort_values(ascending = False).iloc[:vip_num].reset_index()['구매가격'].mean()

15237029.317269076

In [476]:
train_tr.groupby('ID')["구매가격"].skew()

,구매가격
ID,
train_0,-0.656466
train_1,2.651821
train_10,4.154573
train_100,0.945465
train_1000,-0.374373
...,...
train_9995,NaN
train_9996,4.590063
train_9997,-0.330031


In [477]:
# # code_test
# agg_list = [
#         ('vip', lambda x: pd.Series(x.sum()).sort_values(ascending = False).iloc[:vip_num]),

#     ]

# train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()

In [478]:
# code_acting
vip_num = int(train_tr.groupby('ID')['구매가격'].mean().count() * 0.1)
agg_list = [
        ('vip', lambda x: pd.Series(x.sum()).sort_values(ascending = False).iloc[:vip_num]),

        # ('vip_평균구매액', lambda x: np.mean(pd.Series(x.sum()).sort_values(ascending = False).iloc[:vip_num].reset_index()['구매가격']))
    ]

tmp = train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')
train_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,가전제품,공산_식품,생식품,여성_기혼_패션,남성_기혼_패션,아동_대분류,여성_기혼,남녀_기혼,영_아동_대분류,vip
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,0.000000,0.000000,0.050000,0.050000,0.000000,0.0,0.000000,0.150000,0.0,4201200
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,0.000000,0.023810,0.023810,0.214286,0.023810,0.0,0.047619,0.023810,0.0,5043025
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,0.008772,0.000000,0.000000,0.263158,0.043860,0.0,0.289474,0.052632,0.0,22135386
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,0.004739,0.009479,0.004739,0.270142,0.033175,0.0,0.018957,0.000000,0.0,35594762
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,0.000000,0.016129,0.048387,0.370968,0.000000,0.0,0.241935,0.000000,0.0,3988866


In [479]:
tmp = test_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')
test_ft.head()

,ID,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,...,가전제품,공산_식품,생식품,여성_기혼_패션,남성_기혼_패션,아동_대분류,여성_기혼,남녀_기혼,영_아동_대분류,vip
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,0.0,0.000000,0.0,0.285714,0.000000,0.285714,0.0,0.142857,0.285714,429579
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,0.0,0.000000,0.0,0.250000,0.000000,0.000000,0.5,0.000000,0.000000,290600
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,0.0,0.000000,0.0,0.600000,0.200000,0.000000,0.0,0.000000,0.000000,607000
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,0.0,0.000000,0.0,0.250000,0.000000,0.000000,0.0,0.031250,0.000000,2305740
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,0.0,0.047619,0.0,0.095238,0.047619,0.000000,0.0,0.000000,0.000000,1568690


In [480]:
train_ft.shape, test_ft.shape

((14940, 43), (12225, 43))

In [481]:
# train_tr['중분류'].unique()

In [482]:
# train_tr['대분류'].unique()

In [483]:
train_tr[train_tr['대분류'] == '']['중분류'].unique()

array([], dtype=object)

In [484]:
# 피처 아이디어
# 대분류
#   가정용품 : 냉장고+세탁기, 침구, 수입도자기, 주방용품, 취사소형, 욕실용품, 냉난방, 가스렌지
#   골프_유니캐주얼 : 진캐쥬얼, 골프(국내), 트래디셔널, 골프(수입)
#   공산품 > 용기보증, 야채, 건강식품, 일용잡화. 청과
#   남성의류 > 내셔날, 라이센스, 셔츠
#   남성정장스포츠 > 골프웨어, 라이센스, 수입, 내셔날, 스포츠웨어, 임대골프, 골프단품
#   로얄부띠끄 > 수입부띠끄,국내부띠끄, 로얄수입행사
#   로얄부틱 > 엘레강스부틱, 의류기타
#   명품잡화 > 수입종합화장품, 명품, 로얄부틱2F, 보석
#   생식품 > 용기보증, 야채, 정육, 생선, 청과, 곡물, 건식품
#   스포츠캐쥬얼 > 트레디셔널캐주얼, 골프웨어, 트래디셔널캐쥬얼, 골프(LC), 스포츠웨어, 스포츠 슈즈
#   여성의류파트 > 하이캐쥬얼, 디자이너숍, 미시케쥬얼, 레이디숍A, 모피, 디자이너니트, 타운단품
#   여성정장 > 부띠끄, 타운웨어, 디자이너부띠끄, 모피, 엘레강스부틱
#   여성캐주얼 > 캐릭터, 영캐주얼
#   여성캐쥬얼 > 엘레강스, 캐릭터
#   영라이브 > 영캐쥬얼, 진캐쥬얼
#   영어덜트캐쥬얼 > 캐릭터캐쥬얼, 캐릭터슈즈
#   잡화 > 수입종합화장품
#   잡화파트 > 수입종합화장품, 수입의류
#   패션잡화 > 수입종합화장품
# 중분류 > '행사'포함, '모피'포함, 인스탄트, 패스트푸드, 원목(주니어), 보석

# 항상 확인하기
- 학습데이터와 테스트 데이터의 피처개수는 동일해야 함

In [485]:
train_ft.shape, test_ft.shape

((14940, 43), (12225, 43))

In [486]:
train_ft = train_ft.fillna(0)
test_ft = test_ft.fillna(0)

train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [487]:
train_ft.shape, test_ft.shape

((14940, 43), (12225, 43))

In [488]:
train_ft = train_ft.drop(columns="ID")
test_ft = test_ft.drop(columns="ID")

In [489]:
cols = train_ft.select_dtypes("object").columns.tolist()
cols

['주구매지점', '주구매_중분류', '주구매_대분류']

In [490]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown="ignore")

In [491]:
enc.fit_transform(train_ft[cols]).toarray()

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [492]:
tmp = pd.DataFrame(
    enc.fit_transform(train_ft[cols]).toarray(),
    columns =  enc.get_feature_names_out()
)

train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols)
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_대분류_여성캐주얼,주구매_대분류_여성캐쥬얼,주구매_대분류_영라이브,주구매_대분류_영어덜트캐쥬얼,주구매_대분류_영캐릭터,주구매_대분류_영플라자,주구매_대분류_잡화,주구매_대분류_잡화파트,주구매_대분류_케주얼_구두_아동,주구매_대분류_패션잡화
0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,11,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [493]:
test_ft.columns

Index(['내점일수', '구매주기', '주말방문비율', '봄_구매비율', '여름_구매비율', '가을_구매비율', '겨울_구매비율',
       '주구매요일', '일별평균구매건수', '거래개월수', '방문지점수', '주구매지점', '브랜드코드_nunique',
       '중분류_nunique', '주구매_중분류', '대분류_nunique', '주구매_대분류', '총구매액', '구매건수',
       '평균구매액', '최대구매액', '총구매액_왜도', '최소구매액', '환불금액', '환불건수', '구매금액표준편차',
       '퇴근후_구매비율', '2004_어린이날_전날_구매비율', '2004_어린이날_구매비율', '2004_어버이날_전날_구매비율',
       '2004_어버이날_구매비율', '중분류_방문횟수_평균', '가전제품', '공산_식품', '생식품', '여성_기혼_패션',
       '남성_기혼_패션', '아동_대분류', '여성_기혼', '남녀_기혼', '영_아동_대분류', 'vip'],
      dtype='object')

In [494]:
train_ft.shape, test_ft.shape

((14940, 317), (12225, 42))

In [495]:
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols)
test_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_대분류_여성캐주얼,주구매_대분류_여성캐쥬얼,주구매_대분류_영라이브,주구매_대분류_영어덜트캐쥬얼,주구매_대분류_영캐릭터,주구매_대분류_영플라자,주구매_대분류_잡화,주구매_대분류_잡화파트,주구매_대분류_케주얼_구두_아동,주구매_대분류_패션잡화
0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,7,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,4,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [496]:
train_ft.shape, test_ft.shape

((14940, 317), (12225, 317))

In [497]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [498]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_대분류_여성캐주얼,주구매_대분류_여성캐쥬얼,주구매_대분류_영라이브,주구매_대분류_영어덜트캐쥬얼,주구매_대분류_영캐릭터,주구매_대분류_영플라자,주구매_대분류_잡화,주구매_대분류_잡화파트,주구매_대분류_케주얼_구두_아동,주구매_대분류_패션잡화
0,-0.437126,0.002987,0.257728,-1.029777,0.001191,0.838272,0.338186,0.109631,-0.057297,0.017620,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,-0.235019,-0.228606,-0.342432,-0.280675,-0.319269
1,0.239394,-0.356452,-1.008554,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.222706,1.176010,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,4.254981,-0.228606,-0.342432,-0.280675,-0.319269
2,1.890101,-0.869935,0.036742,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,0.277707,1.465608,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,-0.235019,-0.228606,-0.342432,-0.280675,-0.319269
3,4.298510,-1.023980,-0.080558,0.420933,-0.327474,0.008592,-0.135636,0.109631,0.674668,1.465608,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,-0.235019,-0.228606,-0.342432,3.562844,-0.319269
4,0.618244,-0.613193,0.302875,-0.752532,1.707410,-0.130285,-0.821561,0.693965,0.679008,0.886413,...,-0.165008,-0.08084,-0.133603,-0.081258,-0.101046,-0.235019,-0.228606,-0.342432,-0.280675,-0.319269


In [499]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


In [500]:
from sklearn.model_selection import cross_val_score, KFold
cv = KFold(5, shuffle = True, random_state = 42)

In [501]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(random_state=42)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [502]:
scores = cross_val_score(model, train_ft, target, cv = cv, scoring = 'f1_macro', n_jobs = -1)
scores.mean()

0.6914563150580946

In [503]:
model.fit(train_ft,target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015570 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8592
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 159
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995


LGBMClassifier(random_state=42)

In [504]:
pred = model.predict(test_ft)
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [505]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,1.0
...,...,...
12220,test_12220,1.0
12221,test_12221,1.0
12222,test_12222,0.0
12223,test_12223,0.0


# 추출한 피처 저장하기

In [506]:
final_project = '/content/drive/MyDrive/멋사_final_project/'

In [507]:
# train_ft.to_csv(f"{final_project}train_common_DK_2024_10_30_1차.csv",index=False)
# test_ft.to_csv(f"{final_project}test_common_DK_2024_10_30_1차.csv",index=False)

In [508]:
submit.to_csv(f"{final_project}DK_2024_10_30_2차_submit.csv",index=False)